In [1]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.cross_validation import train_test_split


pd.options.mode.chained_assignment = None  # default='warn'


train_list = ['180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
          '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
          '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
          '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
          '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021',\
           '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029','181101',\
             '181030','181031','181101', '181102', '181103','181104',\
             '181105', '181106', '181107', '181108', '181109','181110']

pred_list = ['181105', '181106', '181107', '181108', '181109','181110','181111']


day_all = ['180531','180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
           '180711', '180712', '180713', '180714', '180715',\
           '180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725',\
           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', \
            '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
            '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021','181022',\
            '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
           '181031', '181101', '181102', '181103', '181104', '181105', '181106', '181107', '181108', '181109',\
           '181110', '181111','181112']

c:\program files\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
import json
import lightgbm as lgb

str_cols = ['order_id','job_name',\
            'start_time','end_time',\
            'day_0_bef_end_time']
features_all_merge = pd.read_csv(r'edt_allfeature_score1119.csv')
features_all_merge = features_all_merge['feature'][:53]
#read_cols = list(set(str_cols+features_all))
read_cols = list(set(str_cols+list(features_all_merge)))

In [19]:


 #选取第几轮数据切片
for n_i in range(2,21):
    train_df = []
    print(n_i)
    #训练数据
    for day in train_list:
        print(day,end=' ')   
        day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train\edt_%s.csv'%day,usecols=read_cols)

        #day_all_data = day_all_data[day_all_data.end_time.notnull()]

        #过滤结束日期超过2天的
        day_after_2 = day_all[day_all.index(day)+1]
        day_after_2_int = int('20'+day_after_2+'235959')
        day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
        #day_all_data = day_all_data[day_all_data.end_time.notnull()]

        round_len = day_all_data.shape[0]//20
        train_df.append(day_all_data[n_i*round_len:(n_i+1)*round_len])
       
        del day_all_data
    train = pd.concat(train_df, ignore_index=True)
    print(train.shape)
    
    
    #模型训练---------------------------------------------------------------------------------------
    train_x,test_x,train_y,test_y = train_test_split(train[features_all_merge],train['day_0_bef_end_time'],random_state=33,test_size=0.1)
    
    y_train = train_y.values
    y_test = test_y.values

    X_train = train_x.values
    X_test = test_x.values
    
    lgb_train = lgb.Dataset(X_train, y_train,silent=True)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    
    # specify your configurations as a dict
    params = {'task': 'train',
       'boosting_type': 'gbdt',
       'objective': 'regression',
       'metric':  'l2',
       'num_leaves': 2**8,
       'learning_rate': 0.1,
       'feature_fraction': 0.9,
       'bagging_fraction': 0.9,
       'bagging_freq': 5,
       'verbose': 50,
        'verbose_eval':50}

    #print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=50,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=10,
                    verbose_eval=False)


    joblib.dump(gbm,'./model_save/edt_gbm_%d_of_20.pkl'%n_i)


    del train
    del X_train
    del X_test
    #模型预测-------------------------------------------------------------------------------------------
    for day in pred_list:
        print(day,end=' ')
        #,usecols=read_cols
        every_day_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train\edt_%s.csv'%day,usecols=read_cols)

        every_day_data.index = range(len(every_day_data))

        valid_x_i = every_day_data[features_all_merge]
        valid_y_i = every_day_data['day_0_bef_end_time']

        vlid_pred_i = gbm.predict(valid_x_i, num_iteration=gbm.best_iteration)
        
        every_day_data = every_day_data[['order_id','job_name','start_time','end_time','day_0_bef_end_time']]
        every_day_data['pred_edt'] = vlid_pred_i
        
#         if n_i ==1:
#             every_day_data['day_0_bef_end_time'] = every_day_data['day_0_bef_end_time']
    
        every_day_data.to_csv('.\pred_edt_every_pice\submit_%s_%d.csv'%(day,n_i),index=False)


2
180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 181016 181017 181018 181019 181020 181021 181022 181023 181024 181025 181026 181027 181028 181029 181101 181030 181031 181101 181102 181103 181104 181105 181106 181107 181108 181109 181110 (528685, 58)
181105 181106 181107 181108 181109 181110 181111 3
180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 181016 181017 181018 181019 181020 181021 181022 181023 181

In [4]:
import pandas as pd
for day in pred_list:
    print(day)
    frist_i = pd.read_csv('.\pred_edt_every_pice\submit_%s_1.csv'%day)[['job_name','order_id','pred_edt']]
    for i in range(2,21):
        print(i,end=' ')
        sub_day_i = pd.read_csv('.\pred_edt_every_pice\submit_%s_%d.csv'%(day,i))[['job_name','order_id','pred_edt']]
        frist_i = frist_i.merge(sub_day_i,how='left',on=['job_name','order_id'])
    cols_do = [cl for cl in frist_i.columns if cl not in ('job_name','order_id')]
    frist_i['pred_edt_mean'] = frist_i[cols_do].apply('mean',axis=1)
    frist_i['pred_edt_median'] = frist_i[cols_do].apply('median',axis=1)
    frist_i = frist_i.round(2)
    frist_i.to_csv(r'E:\jupyter\CTM\8submit\submit_day_%s_mean_meadian.csv'%day,index=False)

181105
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 181106
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 181107
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 181108
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 181109
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 181110
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 181111
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 